## Quickstart Guide (Part_1)

## Building a Language Model Application: LLMs

In [6]:
import os 
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")

## LLMs get prediction from a language model

- This is the most basic building block of LangChain. 
- Provides a uniform interface to interact with different large language models. 

In [13]:
from langchain.llms import OpenAI, HuggingFaceHub

- The temperature argument in the OpenAI LLM wrapper is used to control the level of randomness in the generated text.

- A higher temperature value will result in more diverse and unpredictable text, while a lower temperature value will result in more conservative and predictable text.

- The default value for temperature is 1.0, and valid values range from 0.0 to 1.0.

In [14]:
llm = OpenAI(temperature=0.9)  # model_name="text-davinci-003"
text = "What would be a good company name for a company that makes coldring for summer in india with indian spices?"
print(llm(text))



"Spice Chill Co."


- The default Hugging Face Hub inference APIs do not use specialized hardware. They are also not suitable for running larger models like (bigscience/bloom-560m or google/flan-t5-xxl)

- In queries, the Hugging Face models (small, medium) do not check for grammar and spelling mistakes.

#### google/flan-t5-xl (too big for my system) 

In [22]:
# initialize the wrapper around HuggingfaceHub models and call it on some input

hub_llm = HuggingFaceHub(repo_id = "google/flan-t5-small", model_kwargs = {"temperature":0.9, "max_length":64}) # model_name="text-davinci-003"
text = "What would be a good company name for a company that makes cold drink for summer in india with indian spices?"
print(hub_llm(text))

sasa


## Prompt Templates: Mange propts for LLMs

- Instead of hard-coding that text I want to aks, I can use prompt template to manage the prompt.
- Also, when creating application, its not feasible to pass the input directly to LLM.

- It's similar to `python f-strings`.

In [23]:
# python f-string example
framework = "LangChain"
print(f"I'm learning {framework}.")     

I'm learning LangChain.


In [24]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables = ['product'],
    template = "What is a good name for company that makes {product} ?",
)

In [25]:
print(prompt.format(product = "colorful shirts"))

What is a good name for company that makes colorful shirts ?


## Chains: Compbine LLMs and Propts in multi-step workflows

- In real application, Using LLM in isolation is OK for some applications but in most of the cases it requires chaining. And chaining with PromptTemplate might be a neccessity.

- A chain in LangChain is made up of links, which can be either primitives like LLMs or other chains

- Extending the previous example, I can construct an LLMChain which takes user input, formats it with a PromptTemplate, and then passes the formatted response to an LLM.

In [26]:
llm = OpenAI(temperature=0.9) # model_name="text-davinci-003"

prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

In [27]:
from langchain.chains import LLMChain

In [30]:
llm_chain = LLMChain(prompt=prompt, llm = llm)

In [31]:
print(llm_chain.run("colorful shirts"))



Rainbow Threads or Colorful Creations Co.


## Agents: Dynamically call chains based on user input

- So, far what I did was to run the chains in a predetermined order. 

- Agents can use an LLM to determine which actions to take and in what order. An action can either be using a tool and observing its output or returning to the user. 

In order to load agents, understanding the following concepts is crucial. 

- LLM: The language model powering the agent. 

- Tool: A function that performs a specific duty. This can be things like: Google Search, Database lookup, Python REPL, other chains.

- Agents: Agents involve an LLM making decision about which actions to take, taking that action, seeing an observation, and repeating that until its done. 

Agents: for a list of supported agents and their specifications. 

Tools: For a list of predefined tools and their specifications. 

In [33]:
# ! pip install google-search-results

In [35]:
from dotenv import load_dotenv

load_dotenv()

SERPAPI_API_KEY = os.getenv('SERPAPI_API_KEY')

In [38]:
from langchain.agents import load_tools, initialize_agent, AgentType
from langchain.llms import OpenAI

# First, let's load the language model we're going to use to control the agent.
llm = OpenAI(temperature=0)

# Next, let's load some tools to use. Note that the `llm-math` tool uses an LLM, so we need to pass that in.
#tools = load_tools(["serpapi", "llm-math"], llm=llm)
tools = load_tools(["serpapi"])

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Now let's test it out!
agent.run("What is LangChain ?")



> Entering new AgentExecutor chain...
 I should search for LangChain on a search engine to find out more information.
Action: Search
Action Input: LangChain
Observation: {'type': 'organic_result', 'title': 'Things to know'}
Thought: I should click on the first result to learn more.
Action: Search
Action Input: LangChain
Observation: {'type': 'organic_result', 'title': 'Things to know'}
Thought: I should read through the information provided to understand what LangChain is.
Action: Search
Action Input: LangChain
Observation: {'type': 'organic_result', 'title': 'Things to know'}
Thought: I now know the final answer.
Final Answer: LangChain is a blockchain-based platform that aims to revolutionize the language learning industry by providing a decentralized and transparent ecosystem for language learners and teachers.

> Finished chain.


'LangChain is a blockchain-based platform that aims to revolutionize the language learning industry by providing a decentralized and transparent ecosystem for language learners and teachers.'